In [1]:
import pandas as pd
import os

CLINICAL_PATH = "../datasets/clinical_data"

# Create clinical_df DataFrame with age_at_initial_pathologic_diagnosis from XML files:

 ## Clinical data
 
We want to create a DataFrame clinical_df with the following columns:
- **folder_name**: the name of the subfolder in clinical_data
- **file_name**: the name of the XML file
- **age_at_initial_pathologic_diagnosis**: the value from the XML file

In [20]:
def import_xml(file_path):
    temp_df = pd.read_xml(file_path, parser="etree")

    if 'days_to_birth' in temp_df.columns and 'age_at_initial_pathologic_diagnosis' in temp_df.columns:
        temp_df = temp_df[["age_at_initial_pathologic_diagnosis"]]
    
        temp_df = temp_df.dropna(how='all')
        
        if temp_df.empty:
            print(f"File {file_path} has no valid age data.")
        return temp_df
    return None


def main():
    clinical_df = pd.DataFrame(columns=["folder_name", "file_name", "age_at_initial_pathologic_diagnosis"])
    for subfolder in os.listdir(f"{CLINICAL_PATH}"):
        file_xml_or_annotation_or_orm = 0
        for file in os.listdir(f"{CLINICAL_PATH}/{subfolder}"):
            if file.endswith(".xml") and "annotations" not in file and "org_omf" not in file:
                file_xml_or_annotation_or_orm = 1
                selected_data = import_xml(f"{CLINICAL_PATH}/{subfolder}/{file}")
            
                if selected_data is not None:
                    selected_data.insert(0, "file_name", file)
                    selected_data.insert(0, "folder_name", subfolder)

                    # Safe concatenation avoiding empty DataFrames or with NaN values
                    if not selected_data.empty and not selected_data.isna().all().all():
                        clinical_df = pd.concat([clinical_df, selected_data], ignore_index=True)
                        
    return clinical_df

clinical_df = main()
clinical_df.shape

C:\Users\giorg\AppData\Local\Temp\ipykernel_20508\2494993394.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  clinical_df = pd.concat([clinical_df, selected_data], ignore_index=True)


(771, 3)

In [21]:
clinical_df.head()

,folder_name,file_name,age_at_initial_pathologic_diagnosis
0,00049989-fa21-48fb-8dda-710c0dd5932e,nationwidechildrens.org_clinical.TCGA-A2-A0CT.xml,71.0
1,004b6bd4-19d0-4b40-99ef-1a76313fe7a5,nationwidechildrens.org_clinical.TCGA-GM-A2DD.xml,53.0
2,00a5e81c-cd67-483f-9d99-3c733b2ead38,nationwidechildrens.org_clinical.TCGA-D8-A1JM.xml,59.0
3,014f5ae1-5862-4165-9a3b-bba7bb08a527,nationwidechildrens.org_clinical.TCGA-C8-A12P.xml,55.0
4,01a962ea-a87f-49fa-9a27-7273a39f64a9,nationwidechildrens.org_clinical.TCGA-S3-A6ZF.xml,64.0


In [22]:
clinical_df[["age_at_initial_pathologic_diagnosis"]].describe()

,age_at_initial_pathologic_diagnosis
count,771.000000
mean,57.413748
std,13.184850
min,26.000000
25%,48.000000
50%,58.000000
75%,66.000000
max,90.000000


In [23]:
print("-------------------------------------\n"
      "\t\tNaN values check:")
# Check is there are any NaN values in the DataFrame
print(clinical_df.isna().sum())
print("-------------------------------------")

-------------------------------------
		NaN values check:
folder_name                            0
file_name                              0
age_at_initial_pathologic_diagnosis    0
dtype: int64
-------------------------------------


In [24]:
print("-------------------------------------")
# Check if all folder_name values are unique
print(f"-> All folder names are unique? {clinical_df['folder_name'].nunique() == len(clinical_df)}")

# Check if all file_name values are unique
print(f"-> All file names are unique? {clinical_df['file_name'].nunique() == len(clinical_df)}")
print("-------------------------------------")

-------------------------------------
-> All folder names are unique? True
-> All file names are unique? True
-------------------------------------


In [25]:
def find_non_contributing_subfolders():
    all_subfolders = set(os.listdir(f"{CLINICAL_PATH}"))
    
    # Take the subfolders that contributed to clinical_df
    contributing_subfolders = set(clinical_df['folder_name'].unique())
    
    # Folders that did not contribute
    non_contributing_subfolders = all_subfolders - contributing_subfolders
    
    return non_contributing_subfolders

non_contributing_subfolders = find_non_contributing_subfolders()
print("Subfolders don't contribute:", non_contributing_subfolders)
print("\n-----------------------------------------------------")
print(f"-> Do not contribute to the dataframe {len(non_contributing_subfolders)} subfolders.")
print("-----------------------------------------------------")

Subfolders don't contribute: {'1e6b79ff-9787-4cbe-b19d-ebabb6b43589', 'cd7890a5-76f8-479d-9298-fda762a15701', 'dfc14560-ce1a-4d7f-9b03-6777d9ee725a', '100659db-c917-4318-ab86-192cc46a32f3', '967f57a0-b6d6-4b6f-8882-6cb524a461ca', 'ff679dd3-62be-4332-8157-0a83ffb8516a', '20d28c3f-8a04-4473-9b8f-e6e0ab0acd07', '0a40467f-9495-4c5b-b56e-3347a3ee0572', '024bde93-ff69-4d1f-b301-c053e8c594f5', 'c0ab956b-762c-42c5-94cd-f6f7e2fcd6b3', '2ade91c3-a81c-4362-a7fa-5c2aeb22b476', '4a3c7925-3ae9-44ca-b9d9-62ba78bcba33', 'd583e2e8-32e6-4846-ae69-6fb1880075d9', 'aaff8f3d-916c-4510-8ee2-693ca7178b65', '671069e5-4730-4c23-8adf-79576bb1843f', 'e094828e-082a-4da2-9a66-f1172af28288', '62d4515f-a30b-4b1a-b2dd-c8bf9476e803', '5025cc06-de19-4fcc-9c89-c4f7d6450b2c', '5af9bc05-ecb6-4031-9039-a5164bb91aed', '76bf2db2-4aff-4ad8-bb8c-e27525cbe201', '61014733-2fa9-4139-a8ab-5d0dac7c7bac', 'a88c168e-4bba-4bd2-9c0c-77934444cc1c', 'b2235f20-5387-4548-b6d0-7d092d60bf83', '2b09f2e9-68c4-4a64-90c9-d981e89c123d', '415848eb-

**771 rows** = 827 (total subfolders) - 47 (omf files) - 9 (org_clinical_radiation_brca txt files)

# Clinical JSON

In [26]:
df_clinical_json = pd.read_json('../datasets/clinical_data(json&manifest)/files.2024-12-12.json')
print(f"Data cinical JSON shape: {df_clinical_json.shape}")
df_clinical_json.columns

Data cinical JSON shape: (827, 9)


Index(['data_format', 'cases', 'access', 'file_name', 'file_id', 'data_type',
       'data_category', 'file_size', 'annotations'],
      dtype='object')

In [27]:
print(f"Column 'data_category' unique values: {df_clinical_json['data_category'].unique()}")
print(f"Column 'data_format' unique values: {df_clinical_json['data_format'].unique()}")

Column 'data_category' unique values: ['Clinical']
Column 'data_format' unique values: ['BCR XML' 'BCR OMF XML' 'BCR Biotab']


In [28]:
print(df_clinical_json[['cases', 'file_name']].shape)
df_clinical_json[['cases', 'file_name']].head()

(827, 2)


,cases,file_name
0,[{'case_id': 'e4fc0909-f284-4471-866d-d8967b6a...,nationwidechildrens.org_clinical.TCGA-E2-A14P.xml
1,[{'case_id': '87281a89-91d2-44f7-9f80-668567ad...,nationwidechildrens.org_clinical.TCGA-EW-A1J6.xml
2,[{'case_id': 'b8a615f9-d19b-4b09-8ec8-0674e5c6...,nationwidechildrens.org_clinical.TCGA-C8-A12N.xml
3,[{'case_id': '3b01d064-8c00-4972-9f07-407eac8e...,nationwidechildrens.org_clinical.TCGA-BH-A0HX.xml
4,[{'case_id': '8785012f-f73e-4d68-87cf-1d804af3...,nationwidechildrens.org_clinical.TCGA-A7-A13D.xml


In [29]:
# La colonna 'cases' contenga liste di dizionari
cases_expanded = df_clinical_json['cases'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else {})
unique_projects = pd.json_normalize(cases_expanded)['project.project_id'].unique()

print("Column 'cases' is a dict, with project IDs value:", unique_projects)

Column 'cases' is a dict, with project IDs value: ['TCGA-BRCA']


In [30]:
# Extract the 'case_id' value from the list of dictionaries in the 'cases' column
df_clinical_json['case_id'] = df_clinical_json['cases'].apply(
    lambda x: x[0]['case_id'] if isinstance(x, list) and len(x) > 0 and 'case_id' in x[0] else None
)

print(df_clinical_json[['case_id', 'file_name']].head())

                                case_id  \
0  e4fc0909-f284-4471-866d-d8967b6adcbc   
1  87281a89-91d2-44f7-9f80-668567ad5c72   
2  b8a615f9-d19b-4b09-8ec8-0674e5c648cd   
3  3b01d064-8c00-4972-9f07-407eac8e7534   
4  8785012f-f73e-4d68-87cf-1d804af32782   

                                           file_name  
0  nationwidechildrens.org_clinical.TCGA-E2-A14P.xml  
1  nationwidechildrens.org_clinical.TCGA-EW-A1J6.xml  
2  nationwidechildrens.org_clinical.TCGA-C8-A12N.xml  
3  nationwidechildrens.org_clinical.TCGA-BH-A0HX.xml  
4  nationwidechildrens.org_clinical.TCGA-A7-A13D.xml  


# Merge clinical_df with df_clinical_json on file_name

In [31]:
merged_df = pd.merge(clinical_df, df_clinical_json[['case_id', 'file_name']], on='file_name', how='inner')
merged_df.head()

,folder_name,file_name,age_at_initial_pathologic_diagnosis,case_id
0,00049989-fa21-48fb-8dda-710c0dd5932e,nationwidechildrens.org_clinical.TCGA-A2-A0CT.xml,71.0,378778d2-b331-4867-a93b-c64028c8b4c7
1,004b6bd4-19d0-4b40-99ef-1a76313fe7a5,nationwidechildrens.org_clinical.TCGA-GM-A2DD.xml,53.0,b343bfe0-7c23-4c6a-8c84-9ee39db2ecda
2,00a5e81c-cd67-483f-9d99-3c733b2ead38,nationwidechildrens.org_clinical.TCGA-D8-A1JM.xml,59.0,3e775c99-ceda-4246-8d6f-0f58ca5097c8
3,014f5ae1-5862-4165-9a3b-bba7bb08a527,nationwidechildrens.org_clinical.TCGA-C8-A12P.xml,55.0,abdc76db-f85e-4337-a57e-6d098789da03
4,01a962ea-a87f-49fa-9a27-7273a39f64a9,nationwidechildrens.org_clinical.TCGA-S3-A6ZF.xml,64.0,fbee40f1-d6d8-4156-8d42-36e09bb9f095


In [32]:
merged_df.shape

(771, 4)

# miRNA_seq

In [4]:
# Check if all file in all sub folder od datasets/miRNA_seq terminate with mirnas.quantification.txt
count_not_mirnas_quantification_txt = 0
for subfolder in os.listdir("../datasets/miRNA_seq"):
    for file in os.listdir(f"../datasets/miRNA_seq/{subfolder}"):
        if not file.endswith("mirnas.quantification.txt"):
            count_not_mirnas_quantification_txt+=1
            print(f"File {file} in folder {subfolder} does not end with mirnas.quantification.txt")
print(f"Number of files not ending with mirnas.quantification.txt: {count_not_mirnas_quantification_txt}")

File annotations.txt in folder 1d46b67b-8c6d-4a85-a837-30f54d45dce9 does not end with mirnas.quantification.txt
File logs in folder 1d46b67b-8c6d-4a85-a837-30f54d45dce9 does not end with mirnas.quantification.txt
File annotations.txt in folder 2272a72a-201e-4e84-b4fb-5e20939e0fa9 does not end with mirnas.quantification.txt
File logs in folder 2272a72a-201e-4e84-b4fb-5e20939e0fa9 does not end with mirnas.quantification.txt
File annotations.txt in folder 22bbada8-1c32-4c82-8bf0-3d71dce76841 does not end with mirnas.quantification.txt
File logs in folder 22bbada8-1c32-4c82-8bf0-3d71dce76841 does not end with mirnas.quantification.txt
File annotations.txt in folder 26094c76-21b8-4583-aeb4-3bbad0da7f29 does not end with mirnas.quantification.txt
File logs in folder 26094c76-21b8-4583-aeb4-3bbad0da7f29 does not end with mirnas.quantification.txt
File annotations.txt in folder 28c996b2-5e2d-4a48-a4e0-dd910f8599ab does not end with mirnas.quantification.txt
File logs in folder 28c996b2-5e2d-4a

In [34]:
def import_txt(file_path):
    temp_df = pd.read_csv(file_path, sep="\t")
    if 'reads_per_million_miRNA_mapped' in temp_df.columns:
        temp_df = temp_df[["reads_per_million_miRNA_mapped"]]
        temp_df = temp_df.dropna(how='all')
        #print(f"file {file_path}")
        #print(temp_df['reads_per_million_miRNA_mapped'])
        return temp_df
    return None

def miRNA_process():
    count_subfolders = 0
    # Initialize an empty DataFrame to store the results
    miRNA_df1 = pd.DataFrame(columns=["folder_name", "file_name", "reads_per_million_miRNA_mapped"])
    for subfolder in os.listdir("../datasets/miRNA_seq"):
        count_subfolders += 1
        for file in os.listdir(f"../datasets/miRNA_seq/{subfolder}"):
            if file.endswith(".txt") and file != "annotations.xml":
                selected_data = import_txt(f"../datasets/miRNA_seq/{subfolder}/{file}")

                if selected_data is not None:
                    selected_data.insert(0, "file_name", file)
                    selected_data.insert(0, "folder_name", subfolder)

                    # Safe concatenation avoiding empty DataFrames or with NaN values
                    if not selected_data.empty and not selected_data.isna().all().all():
                        miRNA_df1 = pd.concat([miRNA_df1, selected_data], ignore_index=True)
    print(f"Subfolders: {count_subfolders}")
    return miRNA_df1
miRNA_df1 = miRNA_process()

C:\Users\giorg\AppData\Local\Temp\ipykernel_20508\3093818702.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  miRNA_df1 = pd.concat([miRNA_df1, selected_data], ignore_index=True)


Subfolders: 767


**Vengono prese tutte le subfolders**

In [35]:
miRNA_df1.head()

,folder_name,file_name,reads_per_million_miRNA_mapped
0,01626fb9-a7f7-4324-97f2-ef2fee03f3c7,e7b7bf36-aa58-4dc0-8548-a28c11d5060f.mirbase21...,5652.188302
1,01626fb9-a7f7-4324-97f2-ef2fee03f3c7,e7b7bf36-aa58-4dc0-8548-a28c11d5060f.mirbase21...,5872.823005
2,01626fb9-a7f7-4324-97f2-ef2fee03f3c7,e7b7bf36-aa58-4dc0-8548-a28c11d5060f.mirbase21...,5890.158589
3,01626fb9-a7f7-4324-97f2-ef2fee03f3c7,e7b7bf36-aa58-4dc0-8548-a28c11d5060f.mirbase21...,14259.305663
4,01626fb9-a7f7-4324-97f2-ef2fee03f3c7,e7b7bf36-aa58-4dc0-8548-a28c11d5060f.mirbase21...,1534.199167


In [36]:
miRNA_df1.shape

(1442727, 3)

In [37]:
miRNA_df1[["reads_per_million_miRNA_mapped"]].describe()

,reads_per_million_miRNA_mapped
count,1.442727e+06
mean,5.316321e+02
std,8.026859e+03
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,5.006910e-01
max,6.792865e+05


In [38]:
simplified_miRNA_df = miRNA_df1.groupby(
    ['folder_name', 'file_name']
).agg({'reads_per_million_miRNA_mapped': list}).reset_index()

simplified_miRNA_df.shape

(767, 3)

In [39]:
simplified_miRNA_df['list_length'] = simplified_miRNA_df['reads_per_million_miRNA_mapped'].apply(len)
print(simplified_miRNA_df['list_length'].to_list())

[1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 1881, 188

All files have **1881** miRNA values

In [40]:
simplified_miRNA_df.head()

,folder_name,file_name,reads_per_million_miRNA_mapped,list_length
0,01626fb9-a7f7-4324-97f2-ef2fee03f3c7,e7b7bf36-aa58-4dc0-8548-a28c11d5060f.mirbase21...,"[5652.188302, 5872.823005, 5890.158589, 14259....",1881
1,016db033-3cec-4c63-b90f-0428da475a63,f46e1aef-c572-4651-964d-a0a9bc7a1128.mirbase21...,"[8395.099993, 8287.749902, 8330.52851, 20291.5...",1881
2,0173d27e-ff23-42b2-afb9-9b867ace3efc,bc1018a6-62f3-4386-a545-9fa8a31a2e96.mirbase21...,"[19889.387965, 19847.305155, 19961.06025, 1472...",1881
3,0195917c-c127-4523-aff3-9e64ebdd4363,75134cfe-bc9d-4838-9a88-2ef2200b34ec.mirbase21...,"[13142.168087, 13137.975295, 13098.004007, 445...",1881
4,046b15a8-38d3-41fc-b55f-04dd591a8e14,ad7d6f10-1dff-4bc8-a8c3-0b6ce7b45366.mirbase21...,"[10511.654941, 10471.07787, 10569.778855, 2060...",1881


### Check if all miRNA IDs are in the same order in all files

In [11]:
import pandas as pd
import os

def import_txt(file_path):
    temp_df = pd.read_csv(file_path, sep="\t")
    if 'miRNA_ID' in temp_df.columns:
        temp_df = temp_df[["miRNA_ID"]]
        temp_df = temp_df.dropna(how='all')
        return temp_df
    return None

def check_column_order():
    reference_order = None
    mismatches = []

    for subfolder in os.listdir("../datasets/miRNA_seq"):
        for file in os.listdir(f"../datasets/miRNA_seq/{subfolder}"):
            if file.endswith(".txt") and file != "annotations.xml":
                df = import_txt(f"../datasets/miRNA_seq/{subfolder}/{file}")
                if df is not None and not df.empty:
                    current_order = df['miRNA_ID'].tolist()
                    if reference_order is None:
                        reference_order = current_order
                    elif current_order != reference_order:
                        mismatches.append(f"{subfolder}/{file}")

    if len(mismatches) == 0:
        print("---------------------------------------------------")
        print("-> All files have the miRNA IDs in the same order.")
        print("---------------------------------------------------")
    else:
        print("-----------------------------------------------")
        print("-> Files with mismatched miRNA ID order:")
        for f in mismatches:
            print(f)        
        print("-----------------------------------------------")

check_column_order()

--------------------------------------------------
-> All files have the miRNA IDs in the same order.
--------------------------------------------------


# miRNA JSON

In [41]:
df_miRNA_json = pd.read_json('../datasets/miRNA(json&manifest)/files.2024-12-12.json')
print(df_miRNA_json.columns)

Index(['data_format', 'cases', 'access', 'file_name', 'file_id', 'data_type',
       'data_category', 'experimental_strategy', 'platform', 'file_size',
       'annotations'],
      dtype='object')


In [42]:
print(df_miRNA_json[['cases', 'file_name']].head())
print(df_miRNA_json['cases'].head())
print(f"'data_category' unique: {df_miRNA_json['data_category'].unique()}")
print(df_miRNA_json['data_format'].unique())
print(df_miRNA_json.shape)

                                               cases  \
0  [{'case_id': '1c3610f7-e0aa-48d7-9a27-0dbaf6e2...   
1  [{'case_id': '241fffc8-4250-4cfa-b2e7-e68c33ae...   
2  [{'case_id': 'e5aae05a-478e-4a55-a27c-12b2b4be...   
3  [{'case_id': 'b7f74ae1-6f58-447c-be50-a7666eb1...   
4  [{'case_id': 'e7a00d67-2c26-4d1f-bd17-35f659e8...   

                                           file_name  
0  a8c7bebe-a450-4a3e-b891-e2d4ac578b04.mirbase21...  
1  97cb5037-7eea-4ba7-86c3-d13dd91e5b1e.mirbase21...  
2  6bd02673-dd3f-42ed-9997-e371b09f22ee.mirbase21...  
3  f846c0ad-9bbd-46a5-a678-fc3c05cdcc41.mirbase21...  
4  fe712c27-8dbb-4531-afb3-10999e319a7b.mirbase21...  
0    [{'case_id': '1c3610f7-e0aa-48d7-9a27-0dbaf6e2...
1    [{'case_id': '241fffc8-4250-4cfa-b2e7-e68c33ae...
2    [{'case_id': 'e5aae05a-478e-4a55-a27c-12b2b4be...
3    [{'case_id': 'b7f74ae1-6f58-447c-be50-a7666eb1...
4    [{'case_id': 'e7a00d67-2c26-4d1f-bd17-35f659e8...
Name: cases, dtype: object
'data_category' unique: ['Trans

In [43]:
print(df_miRNA_json[['cases', 'file_name']].shape)
df_miRNA_json[['cases', 'file_name']].head(1)

(767, 2)


,cases,file_name
0,[{'case_id': '1c3610f7-e0aa-48d7-9a27-0dbaf6e2...,a8c7bebe-a450-4a3e-b891-e2d4ac578b04.mirbase21...


In [44]:
# La colonna 'cases' contenga liste di dizionari
cases_expanded = df_miRNA_json['cases'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else {})
unique_projects = pd.json_normalize(cases_expanded)['project.project_id'].unique()

print("Unique project IDs:", unique_projects)

Unique project IDs: ['TCGA-BRCA']


In [45]:
# Estrai il valore di 'case_id' dalla lista di dizionari nella colonna 'cases'
df_miRNA_json['case_id'] = df_clinical_json['cases'].apply(
    lambda x: x[0]['case_id'] if isinstance(x, list) and len(x) > 0 and 'case_id' in x[0] else None
)

print(df_miRNA_json[['case_id', 'file_name']].head())

                                case_id  \
0  e4fc0909-f284-4471-866d-d8967b6adcbc   
1  87281a89-91d2-44f7-9f80-668567ad5c72   
2  b8a615f9-d19b-4b09-8ec8-0674e5c648cd   
3  3b01d064-8c00-4972-9f07-407eac8e7534   
4  8785012f-f73e-4d68-87cf-1d804af32782   

                                           file_name  
0  a8c7bebe-a450-4a3e-b891-e2d4ac578b04.mirbase21...  
1  97cb5037-7eea-4ba7-86c3-d13dd91e5b1e.mirbase21...  
2  6bd02673-dd3f-42ed-9997-e371b09f22ee.mirbase21...  
3  f846c0ad-9bbd-46a5-a678-fc3c05cdcc41.mirbase21...  
4  fe712c27-8dbb-4531-afb3-10999e319a7b.mirbase21...  


# Merge miRNA_df1 with df_miRNA_json on file_name

In [46]:
miRNA_merged_df = pd.merge(simplified_miRNA_df, df_miRNA_json[['case_id', 'file_name']], on='file_name', how='inner')
miRNA_merged_df.head()

,folder_name,file_name,reads_per_million_miRNA_mapped,list_length,case_id
0,01626fb9-a7f7-4324-97f2-ef2fee03f3c7,e7b7bf36-aa58-4dc0-8548-a28c11d5060f.mirbase21...,"[5652.188302, 5872.823005, 5890.158589, 14259....",1881,045c13ef-3db7-4adf-b0a3-23338f0479f3
1,016db033-3cec-4c63-b90f-0428da475a63,f46e1aef-c572-4651-964d-a0a9bc7a1128.mirbase21...,"[8395.099993, 8287.749902, 8330.52851, 20291.5...",1881,64089c20-939f-48b4-ae8b-904ad0597145
2,0173d27e-ff23-42b2-afb9-9b867ace3efc,bc1018a6-62f3-4386-a545-9fa8a31a2e96.mirbase21...,"[19889.387965, 19847.305155, 19961.06025, 1472...",1881,db8d38ab-cc61-49ee-863d-40060112f2af
3,0195917c-c127-4523-aff3-9e64ebdd4363,75134cfe-bc9d-4838-9a88-2ef2200b34ec.mirbase21...,"[13142.168087, 13137.975295, 13098.004007, 445...",1881,9495f029-9882-4c9b-a557-b3aec9e91698
4,046b15a8-38d3-41fc-b55f-04dd591a8e14,ad7d6f10-1dff-4bc8-a8c3-0b6ce7b45366.mirbase21...,"[10511.654941, 10471.07787, 10569.778855, 2060...",1881,97b5f490-1cd9-4833-b903-5f4c6e45670d


In [47]:
miRNA_merged_df.shape

(767, 5)

# Merge the two merged DataFrames on case_id

In [48]:
final_merged_df = pd.merge(merged_df, miRNA_merged_df, on='case_id', suffixes=('_clinical', '_miRNA'))
final_merged_df.head()

,folder_name_clinical,file_name_clinical,age_at_initial_pathologic_diagnosis,case_id,folder_name_miRNA,file_name_miRNA,reads_per_million_miRNA_mapped,list_length
0,00049989-fa21-48fb-8dda-710c0dd5932e,nationwidechildrens.org_clinical.TCGA-A2-A0CT.xml,71.0,378778d2-b331-4867-a93b-c64028c8b4c7,b02ef951-bb51-4378-b59c-ccffeff088ec,ce7471cc-a727-42bb-bded-e4a657e5b8fb.mirbase21...,"[12120.990742, 12041.557881, 12141.51573, 1768...",1881
1,004b6bd4-19d0-4b40-99ef-1a76313fe7a5,nationwidechildrens.org_clinical.TCGA-GM-A2DD.xml,53.0,b343bfe0-7c23-4c6a-8c84-9ee39db2ecda,05960e82-e648-4b99-a116-9cf917d1bff7,d042b37f-1ba7-41e8-af11-9232df313018.mirbase21...,"[20904.21904, 21106.26856, 20907.030164, 45794...",1881
2,004b6bd4-19d0-4b40-99ef-1a76313fe7a5,nationwidechildrens.org_clinical.TCGA-GM-A2DD.xml,53.0,b343bfe0-7c23-4c6a-8c84-9ee39db2ecda,9326bb7f-da05-490c-a620-5c88323130d6,11c5ea50-2bce-4da6-b76f-548b9f64238f.mirbase21...,"[4742.042712, 4748.951756, 4919.239371, 4809.9...",1881
3,00a5e81c-cd67-483f-9d99-3c733b2ead38,nationwidechildrens.org_clinical.TCGA-D8-A1JM.xml,59.0,3e775c99-ceda-4246-8d6f-0f58ca5097c8,4e60fda1-4bbe-4901-987d-6c15a87af886,397fe4a8-e6f0-4b36-88e0-dcbb488eff59.mirbase21...,"[8992.901265, 8985.700823, 9044.654437, 29490....",1881
4,014f5ae1-5862-4165-9a3b-bba7bb08a527,nationwidechildrens.org_clinical.TCGA-C8-A12P.xml,55.0,abdc76db-f85e-4337-a57e-6d098789da03,5cc8d0d0-e196-4ef2-85c8-20f8d35764e3,004b1938-87ab-4bf2-a19b-d2953d0e81a0.mirbase21...,"[6540.401419, 6668.261153, 6475.719437, 19448....",1881


In [49]:
final_merged_df.shape

(758, 8)

In [50]:
final_merged_df.isna().sum()

folder_name_clinical                   0
file_name_clinical                     0
age_at_initial_pathologic_diagnosis    0
case_id                                0
folder_name_miRNA                      0
file_name_miRNA                        0
reads_per_million_miRNA_mapped         0
list_length                            0
dtype: int64

In [51]:
final_merged_df.columns

Index(['folder_name_clinical', 'file_name_clinical',
       'age_at_initial_pathologic_diagnosis', 'case_id', 'folder_name_miRNA',
       'file_name_miRNA', 'reads_per_million_miRNA_mapped', 'list_length'],
      dtype='object')

## Righe non mergiate

In [52]:
# Merge con outer join per includere tutte le righe
all_rows_df = pd.merge(merged_df, miRNA_merged_df, on='case_id', suffixes=('_clinical', '_miRNA'), how='outer')

# Filtra le righe senza corrispondenza di case_id
non_matching_rows_df = all_rows_df[all_rows_df.isna().any(axis=1)]

non_matching_rows_df.shape

(66, 8)

In [53]:
non_matching_rows_df.head(10)

,folder_name_clinical,file_name_clinical,age_at_initial_pathologic_diagnosis,case_id,folder_name_miRNA,file_name_miRNA,reads_per_million_miRNA_mapped,list_length
41,174eca80-6a0a-4f82-a87f-d47a117f8b91,nationwidechildrens.org_clinical.TCGA-E2-A15E.xml,40.0,0a2a3529-f645-4967-9a58-89ee20b8bb62,NaN,NaN,NaN,NaN
46,07442715-1af6-42b0-801b-84fc7d01c1a0,nationwidechildrens.org_clinical.TCGA-C8-A1HG.xml,50.0,0c23c380-363c-474d-b64c-b47f612a8225,NaN,NaN,NaN,NaN
69,77155c92-4416-4f53-9435-776ceee32845,nationwidechildrens.org_clinical.TCGA-D8-A27F.xml,40.0,1549dc64-3dab-43fc-96e9-b07d520957e1,NaN,NaN,NaN,NaN
73,5038627b-4066-4919-8a89-8bc55143596c,nationwidechildrens.org_clinical.TCGA-E9-A22H.xml,42.0,161917b8-88ad-407b-ade6-d6b98478b359,NaN,NaN,NaN,NaN
77,492170e4-90b8-4560-89ff-effa9eeb5194,nationwidechildrens.org_clinical.TCGA-B6-A0X1.xml,48.0,178b2c48-c07d-422e-ae17-8bcfd996ad51,NaN,NaN,NaN,NaN
78,63e03017-0ea7-4a4d-85ee-32df750e68a8,nationwidechildrens.org_clinical.TCGA-A7-A0D9.xml,37.0,17baef7c-d97d-4b98-ab53-503ef856523d,NaN,NaN,NaN,NaN
84,57f570ac-1320-480e-a7b0-fffd2e882513,nationwidechildrens.org_clinical.TCGA-A2-A0SX.xml,48.0,18d35983-ea6a-4b70-a209-9bef37595956,NaN,NaN,NaN,NaN
99,7618fafa-1164-40cb-b464-6ac90ad35c60,nationwidechildrens.org_clinical.TCGA-E2-A15J.xml,51.0,1fef9454-b3e8-4d9d-a9aa-aa1f1a32b80c,NaN,NaN,NaN,NaN
141,58b8a73e-3975-43ec-995a-e8a3d72f3930,nationwidechildrens.org_clinical.TCGA-A7-A13F.xml,44.0,2cf68894-168b-458b-af4f-53cad72989a8,NaN,NaN,NaN,NaN
220,d1a2492b-b588-4adc-9f0f-4fc83a8cf0d3,nationwidechildrens.org_clinical.TCGA-AR-A1AN.xml,46.0,43029932-feb0-4658-97fa-645c1e775009,NaN,NaN,NaN,NaN


# Save in csv file

In [54]:
# Select relevant columns to save
final_merged_df_to_save = final_merged_df[['case_id', 'age_at_initial_pathologic_diagnosis', 'reads_per_million_miRNA_mapped']]
final_merged_df_to_save.to_csv('../datasets/preprocessed/final_merged_clinical_miRNA.csv', index=False)

In [57]:
final_merged_df = final_merged_df_to_save.copy()
# Espandi la lista in colonne separate
reads_df = pd.DataFrame(final_merged_df["reads_per_million_miRNA_mapped"].tolist()).fillna(0)

# Rinomina le colonne come miRNA_1, miRNA_2, ...
reads_df.columns = [f"miRNA_{i+1}" for i in range(reads_df.shape[1])]

# Unisci con le altre colonne (case_id, età, ecc.)
df_finale = pd.concat([final_merged_df.drop(columns=["reads_per_million_miRNA_mapped"]), reads_df], axis=1)

# Visualizza il risultato
print(df_finale.head())
df_finale.to_csv('../datasets/preprocessed/final_merged.csv', index=False)

                                case_id  age_at_initial_pathologic_diagnosis  \
0  378778d2-b331-4867-a93b-c64028c8b4c7                                 71.0   
1  b343bfe0-7c23-4c6a-8c84-9ee39db2ecda                                 53.0   
2  b343bfe0-7c23-4c6a-8c84-9ee39db2ecda                                 53.0   
3  3e775c99-ceda-4246-8d6f-0f58ca5097c8                                 59.0   
4  abdc76db-f85e-4337-a57e-6d098789da03                                 55.0   

        miRNA_1       miRNA_2       miRNA_3       miRNA_4      miRNA_5  \
0  12120.990742  12041.557881  12141.515730  17683.610218  2483.291559   
1  20904.219040  21106.268560  20907.030164  45794.611568  6957.179909   
2   4742.042712   4748.951756   4919.239371   4809.913909   546.220895   
3   8992.901265   8985.700823   9044.654437  29490.757783  3047.586817   
4   6540.401419   6668.261153   6475.719437  19448.969676  2930.996358   

      miRNA_6      miRNA_7      miRNA_8  ...  miRNA_1872  miRNA_1873  \
0 

In [56]:
final_merged_df_to_save.shape

(758, 3)